In [18]:
import pandas as pd 
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk import ngrams
from nltk.corpus import stopwords
import re
nltk.download('punkt')
nltk.download('stopwords') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df= pd.read_csv('train.csv')

In [3]:
#toxic_classification for checking the balance
class_columns = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
df['toxic_classification'] = df[class_columns].max(axis=1)

In [4]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxic_classification
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


In [5]:
#lenght 
df['comment_length'] = df['comment_text'].apply(lambda x: len(str(x)))

In [6]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxic_classification,comment_length
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,264
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,112
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,233
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,622
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,67


In [19]:
#frequent words 
# Function to tokenize and count words
def count_words(texts):
    stop_words = set(stopwords.words('english'))  # Set of English stopwords
    all_words = []
    
    for text in texts:
        # Tokenize and convert to lowercase
        tokens = word_tokenize(text.lower())
        
        # Remove stopwords and special characters
        cleaned_tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens if token not in stop_words and token.isalnum()]
        
        # Extend the list of all words
        all_words.extend(cleaned_tokens)
    
    word_counts = Counter(all_words)
    return word_counts

# Separate comments by class
toxic_comments = df[df['toxic'] == 1]['comment_text']
non_toxic_comments = df[df['toxic'] == 0]['comment_text']

# Count word frequencies for toxic and non-toxic comments
toxic_word_counts = count_words(toxic_comments)
non_toxic_word_counts = count_words(non_toxic_comments)

# Get the most common words and their frequencies
n = 20  # Number of most common words to include
toxic_common_words = toxic_word_counts.most_common(n)
non_toxic_common_words = non_toxic_word_counts.most_common(n)

In [21]:
#create dataset for frequent words
# Create a DataFrame for Tableau
data = {
    'Word': [],
    'Frequency': [],
    'Class': []
}

# Add toxic common words to data
for word, freq in toxic_common_words:
    data['Word'].append(word)
    data['Frequency'].append(freq)
    data['Class'].append('Toxic')

# Add non-toxic common words to data
for word, freq in non_toxic_common_words:
    data['Word'].append(word)
    data['Frequency'].append(freq)
    data['Class'].append('Non-Toxic')

# Create DataFrame
df_word_cloud = pd.DataFrame(data)

#save new dataset
#"""

In [9]:
# Function to calculate sentiment polarity and subjectivity
def analyze_sentiment(text):
    blob = TextBlob(str(text))
    return blob.sentiment.polarity, blob.sentiment.subjectivity

# Apply sentiment analysis to each comment
df['sentiment_polarity'], df['sentiment_subjectivity'] = zip(*df['comment_text'].apply(analyze_sentiment))

In [10]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxic_classification,comment_length,sentiment_polarity,sentiment_subjectivity
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,264,0.136364,0.454545
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,112,0.287500,0.550000
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,233,0.160000,0.406667
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,622,0.200000,0.301042
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,67,0.000000,0.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      159571 non-null  object 
 1   comment_text            159571 non-null  object 
 2   toxic                   159571 non-null  int64  
 3   severe_toxic            159571 non-null  int64  
 4   obscene                 159571 non-null  int64  
 5   threat                  159571 non-null  int64  
 6   insult                  159571 non-null  int64  
 7   identity_hate           159571 non-null  int64  
 8   toxic_classification    159571 non-null  int64  
 9   comment_length          159571 non-null  int64  
 10  sentiment_polarity      159571 non-null  float64
 11  sentiment_subjectivity  159571 non-null  float64
dtypes: float64(2), int64(8), object(2)
memory usage: 14.6+ MB


In [12]:
columns= ['toxic_classification','comment_length','sentiment_polarity','sentiment_subjectivity']
new_df= df[columns]

In [13]:
# Define and customize stop words if needed
stop_words = set(stopwords.words('english'))

# Function to tokenize text, remove stop words, and extract n-grams
def extract_ngrams(texts, n):
    ngram_counts = Counter()
    for text in texts:
        tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
        cleaned_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]  # Remove stop words and non-alphanumeric tokens
        ngrams_list = list(ngrams(cleaned_tokens, n))  # Generate n-grams
        ngram_counts.update(ngrams_list)  # Update counter with n-grams
    return ngram_counts

# Example: Extract and count bigrams (n=2) after stop word removal
bigram_counts = extract_ngrams(df['comment_text'], 2)

# Example: Extract and count trigrams (n=3) after stop word removal
trigram_counts = extract_ngrams(df['comment_text'], 3)

# Convert n-gram counts to DataFrame for easier handling
bigram_df = pd.DataFrame(bigram_counts.most_common(), columns=['Bigram', 'Frequency'])
trigram_df = pd.DataFrame(trigram_counts.most_common(), columns=['Trigram', 'Frequency'])

# Save the most common n-grams to CSV files
#bigram_df.to_csv('most_common_bigrams.csv', index=False)
#trigram_df.to_csv('most_common_trigrams.csv', index=False)

In [ ]:
def extract_bigrams_by_class(data, class_column, text_column, class_value):
    class_data = data[data[class_column] == class_value][text_column]
    bigram_counts = Counter()
    for text in class_data:
        tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
        cleaned_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]  # Remove stop words and non-alphanumeric tokens
        bigrams_list = list(ngrams(cleaned_tokens, 2))  # Generate bigrams
        bigram_counts.update(bigrams_list)  # Update counter with bigrams
    return bigram_counts


# Example: Extract and count bigrams for toxic comments
bigram_counts_toxic = extract_bigrams_by_class(df, 'class_column', 'comment_text', 'toxic')

# Example: Extract and count bigrams for non-toxic comments
bigram_counts_non_toxic = extract_bigrams_by_class(df, 'class_column', 'comment_text', 'non-toxic')

# Convert bigram counts to DataFrames for easier handling
bigram_df_toxic = pd.DataFrame(bigram_counts_toxic.most_common(), columns=['Bigram', 'Frequency'])
bigram_df_non_toxic = pd.DataFrame(bigram_counts_non_toxic.most_common(), columns=['Bigram', 'Frequency'])


In [ ]:
import matplotlib.pyplot as plt

# Select top 10 bigrams for toxic comments
top_10_bigrams_toxic = bigram_df_toxic.head(10)

# Plotting for toxic comments
plt.figure(figsize=(10, 6))
plt.barh(top_10_bigrams_toxic['Bigram'], top_10_bigrams_toxic['Frequency'], color='red')
plt.xlabel('Frequency')
plt.ylabel('Bigram')
plt.title('Top 10 Bigrams - Toxic Comments')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest frequency on top
plt.show()

# Select top 10 bigrams for non-toxic comments
top_10_bigrams_non_toxic = bigram_df_non_toxic.head(10)

# Plotting for non-toxic comments
plt.figure(figsize=(10, 6))
plt.barh(top_10_bigrams_non_toxic['Bigram'], top_10_bigrams_non_toxic['Frequency'], color='green')
plt.xlabel('Frequency')
plt.ylabel('Bigram')
plt.title('Top 10 Bigrams - Non-Toxic Comments')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest frequency on top
plt.show()


In [24]:
new_df.head(100)

,toxic_classification,comment_length,sentiment_polarity,sentiment_subjectivity
0,0,264,0.136364,0.454545
1,0,112,0.287500,0.550000
2,0,233,0.160000,0.406667
3,0,622,0.200000,0.301042
4,0,67,0.000000,0.000000
...,...,...,...,...
95,0,554,0.143571,0.401905
96,0,55,0.400000,0.550000
97,0,136,0.500000,0.500000
98,0,174,0.200000,0.200000


In [16]:
#save df to csv for tableu
new_df.to_csv('df_eda.csv', index=True)

In [15]:
#df_word_cloud.to_csv('word_frequency_data.csv', index=False)
#bigram_df.to_csv('most_common_bigrams.csv', index=False)
#trigram_df.to_csv('most_common_trigrams.csv', index=False)

In [22]:
df_word_cloud.to_csv('word_frequency_data.csv', index=False)